In [ ]:
!pip install -q --upgrade keras-cv

In [ ]:
import pprint
import os
from tqdm import tqdm
import tensorflow as tf
import keras
import keras_cv
from keras_cv import visualization

In [ ]:
# Dataset a usar para entrenar

# Rutas a los directorios con las imágenes y con las anotaciones de los bounding boxes
# Por cada archivo con una imagen en el directorio de imágenes, tiene que haber otro con el mismo nombre
# en el directorio labels, el cual contiene los bounding box presentes en la imagen.
# Ejemplo: "Perro.jpg" en imágenes y "Perro.txt" en labels.

path_images = "./small/images/"
path_labels = "./small/labels/"

# Archivo con los nombres de clases
class_name_file = "classes_names.txt"

# Extensión de archivo de las imágenes, si son más de una se pueden poner en una tupla. Ejemplo (".jpg", ".png", ".bmp")
image_file_extension = ".jpg"

# Extensión de archivo de las anotaciones, si son más de una se pueden poner en una tupla. Ejemplo (".txt", ".xml", ".json")
label_file_extension = ".txt"

# Formato de las anotaciones:
# Más formatos en https://keras.io/api/keras_cv/bounding_box/
bb_format = "rel_xywh"

# Proporción de muestras para el dataset de entrenamiento
SPLIT_RATIO = 0.7

# Tamaño del batch
BATCH_SIZE = 4

# Velocidad de aprendizaje
LEARNING_RATE = 0.001

In [ ]:
# Leemos las clases de objetos a detectar

file_classes_names = open(class_name_file)
classes_names = []
for l in file_classes_names:
  classes_names.append(l.strip())
file_classes_names.close()

class_dict = dict(zip(range(len(classes_names)), classes_names))

pprint.pprint(class_dict)

In [ ]:
# Leemos los archivos de imágenes y sus anotaciones

labels_files = sorted(
    [
        os.path.join(path_labels, file_name)
        for file_name in os.listdir(path_labels)
        if file_name.endswith(label_file_extension)
    ]
)

images_files = sorted(
    [
        os.path.join(path_images, file_name)
        for file_name in os.listdir(path_images)
        if file_name.endswith(image_file_extension)
    ]
)

In [ ]:
# Leemos la información de las anotaciones

def parse_annotation(fn):
  if fn.endswith(".txt"):
    # El archivo debe tener todas las anotaciones que posea la imagen correspondiente,
    # y todas tienen que tener el mismo formato

    _file = open(fn)
    boxes = []
    class_ids = []
    for l in _file:
      data = l.strip().split(" ")

      class_ids.append(int(data[0]))

      xmin = float(data[1])
      ymin = float(data[2])
      width = float(data[3])
      height = float(data[4])
      boxes.append([xmin, ymin, width, height])
    _file.close()

  else:
    # Implementar la lectura desde otros archivos
    pass

  return boxes, class_ids

image_paths = []
bbox = []
classes = []
i = 0
for l in labels_files:
    boxes, class_ids = parse_annotation(l)
    image_path = images_files[i]; i+= 1
    image_paths.append(image_path)
    bbox.append(boxes)
    classes.append(class_ids)

bbox = tf.ragged.constant(bbox)
classes = tf.ragged.constant(classes)
image_paths = tf.ragged.constant(image_paths)

In [ ]:
# Construir el dataset
data = tf.data.Dataset.from_tensor_slices((image_paths, classes, bbox))

# Separar en train set y test set
nsamples = int(len(labels_files) * SPLIT_RATIO)
train_data = data.take(nsamples)
test_data = data.skip(nsamples)

# Leer las imágenes
def load_dataset(image_path, classes, bbox):
    image = tf.io.read_file(image_path)

    # Si no son imágenes jpg entonces leerlas con la función correspondiente
    image = tf.image.decode_jpeg(image, channels=3)

    bounding_boxes = {
        "classes": classes,
        "boxes": bbox,
    }
    return {"images": tf.cast(image, tf.float32), "bounding_boxes": bounding_boxes}

def dict_to_tuple(inputs):
    return inputs["images"], inputs["bounding_boxes"]

train_ds = train_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(BATCH_SIZE * 4)
train_ds = train_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)

augmenter = keras.Sequential(
    layers=[
        keras_cv.layers.RandomFlip(mode="horizontal", bounding_box_format=bb_format),
        keras_cv.layers.RandomShear(
            x_factor=0.2, y_factor=0.2, bounding_box_format=bb_format
        ),
        keras_cv.layers.JitteredResize(target_size=(640, 640), scale_factor=(0.75, 1.3), bounding_box_format=bb_format)
    ]
)
train_ds = train_ds.map(augmenter, num_parallel_calls=tf.data.AUTOTUNE)

train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

test_ds = test_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.shuffle(BATCH_SIZE * 4)
test_ds = test_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)

resizing = keras_cv.layers.JitteredResize(target_size=(640, 640), scale_factor=(0.75, 1.3), bounding_box_format=bb_format)
test_ds = test_ds.map(resizing, num_parallel_calls=tf.data.AUTOTUNE)

test_ds = test_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
# Visualizamos algunas imágenes del dataset

def visualize_dataset(inputs, value_range, rows, cols, bounding_box_format):
    inputs = next(iter(inputs.take(1)))
    images, bounding_boxes = inputs[0], inputs[1]
    visualization.plot_bounding_box_gallery(
        images,
        value_range=value_range,
        rows=rows,
        cols=cols,
        y_true=bounding_boxes,
        scale=5,
        font_scale=0.7,
        bounding_box_format=bounding_box_format,
        class_mapping=class_dict
    )

visualize_dataset(train_ds, bounding_box_format=bb_format, value_range=(0, 255), rows=2, cols=2)

visualize_dataset(test_ds, bounding_box_format=bb_format, value_range=(0, 255), rows=2, cols=2)

In [ ]:
# Cargamos un modelo preentrenado

backbone = keras_cv.models.YOLOV8Backbone.from_preset("yolo_v8_s_backbone_coco")
yolo = keras_cv.models.YOLOV8Detector(
    num_classes=len(class_dict),
    bounding_box_format=bb_format,
    backbone=backbone,
    fpn_depth=1,
)

optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

yolo.compile(optimizer=optimizer, classification_loss="binary_crossentropy", box_loss="ciou")

In [ ]:
# Entrenar el modelo
EPOCHS = 2

yolo.fit(
    train_ds,
    validation_data=test_ds,
    epochs=EPOCHS
)